In [11]:
import matplotlib.colors as mcolors
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer

from interpreto.attributions.methods import IntegratedGradients, OcclusionExplainer, Saliency, SmoothGrad
from interpreto.commons.granularity import GranularityLevel
from interpreto.visualizations.attributions.classification_highlight import (
    GenerationAttributionVisualization,
    MultiClassAttributionVisualization,
    SingleClassAttributionVisualization,
)

Currents:
- Occlusion
- Saliency
- Integrated Gradients
- SmoothGrad

Soon:
- LIME
- KernelSHAP
- Sobol



# Classification task

In [12]:
model_name = "textattack/bert-base-uncased-imdb"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Occlusion 

In [13]:
explainer = OcclusionExplainer(
    model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.TOKEN
)

attribution_outputs = explainer.explain(
    model_inputs=[
        "This is the best movie I have ever seen. The cinematography was uncharacteristically breathtaking."
    ],
    mode="logits",
)

viz = SingleClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

# mode in {"logits", "softmax", "log_softmax"}

In [23]:
explainer = OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD)

list_attribution_outputs = explainer.explain(
    model_inputs=[
        "This is the best movie I have ever seen.",
        "I hate this movie.",
        "This movie is super good. I love it.",
    ],
    mode="logits",
)

for attribution_outputs in list_attribution_outputs:
    viz = SingleClassAttributionVisualization(
        attribution_output=attribution_outputs,
        color=mcolors.to_rgb("red"),
        css=".common-word-style { margin-right: 0.3em }",
    )
    viz.display()

In [6]:
explainer = OcclusionExplainer(
    model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.TOKEN
)

attribution_outputs = explainer.explain(
    model_inputs="This is the best movie I have ever seen.",
    targets=[0, 1],
    mode="logits",
)

viz = MultiClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    class_colors=[mcolors.to_rgb("green"), mcolors.to_rgb("blue")],
    class_names=["negative review", "positive review"],
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

## Saliency

In [5]:
explainer = Saliency(model=model, batch_size=4, tokenizer=tokenizer)

attribution_outputs = explainer.explain(
    model_inputs=["This is the best movie I have ever seen."],
)

viz = SingleClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

## Integrated Gradient

In [20]:
explainer = IntegratedGradients(model=model, tokenizer=tokenizer, batch_size=4, n_interpolations=10)

attribution_outputs = explainer.explain(model_inputs="This is the best movie I have ever seen.")

In [ ]:
viz = SingleClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

## SmoothGrad

In [7]:
explainer = SmoothGrad(model=model, batch_size=4, tokenizer=tokenizer, n_interpolations=50, noise_level=0.01)

attribution_outputs = explainer.explain(
    model_inputs=["I love this movie"],
)

viz = SingleClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

# Generation task

In [14]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

## Occlusion

In [ ]:
explainer = OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD)

attribution_outputs = explainer.explain(
    model_inputs="Hi there, how are you?", mode="softmax", generation_kwargs={"max_length": 10}
)

viz = GenerationAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("orange"),
    highlight_border=False,
    normalize=True,
)
viz.display()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/fanny.jourdan/dev/interpreto/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [9]:
explainer = OcclusionExplainer(
    model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.ALL_TOKENS
)

attribution_outputs = explainer.explain(
    model_inputs="Hi there, how are you?",
    targets="I am fine, thank you",
    mode="softmax",
    generation_kwargs={"max_length": 10},
)

viz = GenerationAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("orange"),
    highlight_border=False,
    normalize=True,
)
viz.display()

## Integrated Gradients

In [ ]:
explainer = IntegratedGradients(model=model, tokenizer=tokenizer, batch_size=4, n_interpolations=10)

list_attribution_outputs = explainer.explain(
    model_inputs=["Hi there, how are you?", "What time is it?"], generation_kwargs={"max_length": 10}
)

for attribution_outputs in list_attribution_outputs:
    viz = GenerationAttributionVisualization(
        attribution_output=attribution_outputs,
        color=mcolors.to_rgb("orange"),
        highlight_border=False,
        normalize=True,
    )
    viz.display()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


/home/fanny.jourdan/dev/interpreto/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
